Импортируем библиотеку pandas. Создадим БД из файла auto.czv. Выведем результат

In [ ]:
import pandas as pd

df = pd.read_csv('auto.csv');
print(df);

Получим информацию о первых 20 строках БД с помощью метода head. Передадим ему число строк, нужных для вывода

In [ ]:
print(df.head(20));

Данные представляют собой информацию о подержанных автомобилях, выставленных на продажу. В таблице содержатся следующие столбцы:
Name - Название и модель автомобиля;
year - Год выпуска автомобиля;
SellingPrice - Цена продажи автомобиля в рублях;
kmdriven - Пробег автомобиля в километрах;
fuel - Тип топлива, используемый автомобилем;
seller_Type - Тип продавца;
transmission - Тип коробки передач;
owner - Количество предыдущих владельцев.
Предметная область данных — вторичный рынок автомобилей. На основе этих данных можно анализировать цены на подержанные автомобили в зависимости от их характеристик, таких как год выпуска, пробег, тип топлива и количество владельцев. Это полезно для покупателей, которые хотят оценить, насколько справедлива цена, и для продавцов, которые хотят определить рыночную стоимость своих автомобилей

Получим информацию по столбцам с помощью info

In [ ]:
print(df.info());

Выведем информацию по числовым данным, используя метод describe

In [ ]:
print(df.describe());

Исследуем названия столбцов при помощи colums

In [ ]:
print(df.columns);

Заметим, что часть названий отличается от общего вида. Для улучшения читаемости приведем их к стилю snake_case. Используем rename

In [ ]:
df = df.rename(columns={'Name': 'name', 'SellingPrice': 'selling_price', 'seller_Type': 'seller_type'})
print(df.columns);

Проверим результат

Перейдем к поиску пропущенных значений. Посчитаем найденные методом isna пропуски с помощью sum

In [ ]:
print(df.isna().sum());

Избавимся от строк с неполными данными в полях selling_price, kmdriven, fuel и посмотрим на итог

In [ ]:
df = df.dropna(subset=['selling_price', 'kmdriven', 'fuel']);
print(df.isna().sum());

Выявим дубликаты. Начнем с явных. Применим метод duplicated в связке с sum

In [ ]:
print(df.duplicated().sum());

Отчистим БД от явных дубликатов методом drop_duplicates. Обновим индексацию, чтобы избавиться от пропусков с помощью reset_index. Снова увидим количество явных дубликатов. Убедимся в их отсутствии

In [ ]:
df = df.drop_duplicates().reset_index(drop=True);
print(df.duplicated().sum());

Перейдем к неяным дубликатам. Вернем перечень уникальных значений столбца name. Заметим, что столбец состоит почти полностью из уникальных данных, поэтому перейдем к следующим полям

In [ ]:
y = df['name'].unique();
for i in range(len(y)):
    print(y[i]);

Исследуем уникальные значения остальных текстовых полей

In [ ]:
print(df['year'].unique());
print(df['fuel'].unique());
print(df['seller_type'].unique());
print(df['transmission'].unique());
print(df['owner'].unique());

Найдем неявные дубликаты в виде опечаток в уже существующих значениях. Заменим эти данных в столбцах fuel и seller_type методом replace

In [ ]:
df['fuel'] = df['fuel'].replace('Petroll','Petrol');
print(df['fuel'].unique());

In [ ]:
df['seller_type'] = df['seller_type'].replace('Dealeer','Dealer');
print(df['seller_type'].unique());

Исследуем уникальные значения числовых полей

In [ ]:
print(df['selling_price'].unique());
print(df['kmdriven'].unique());

Поменяем тип данных на int, так как все значения в выбранных полях целые

In [ ]:
df['selling_price'] = df.selling_price.astype(int);
df['kmdriven'] = df.kmdriven.astype(int);
print(df.info());

Перейдем к группировке. Сгруппируем тип коробки передач (transmission) по количеству разных типов топлива (fuel), используя groupby

In [ ]:
print(df.groupby(['transmission', 'fuel'])['fuel'].count());

Создадим датафрейм из группы, поменяем название последнего поля на count

In [ ]:
group = df.groupby(['transmission', 'fuel'])['fuel'].count().reset_index(name='count');
print(group);

Перейдем к сортировке. Отсортируем датафрейм по убыванию столбца count

In [ ]:
print(group.sort_values(by='count', ascending=False));

Создадим сводные таблицы, используя метод pivot_table. Первая таблица основана на средней стоимости для каждого года year. Отсортирована по
убыванию .Значения округлень до трёх знаокв.

In [ ]:
print(df.pivot_table(index='year', values='selling_price', aggfunc='mean').round(3).sort_values(by='selling_price', ascending=False));

Вторая таблица основана на среднем количестве пройденных километров для каждого типа коробки передач transmission - строки и типа топлива - fuel - столбцы. Отсортирована по возрастанию transmission. Значения округлены до двух знаков

In [ ]:
print(df.pivot_table(index='transmission', columns='fuel', values='kmdriven', aggfunc='mean').round(2).sort_values(by='transmission'));